# Team Ken+digimagi's solution

This notebook is not our final/best model, but  it contains the main ideas of our team.

Special thanks to WeKick.

You can see the episode of the agent that is almost same as this notebook here.
https://www.kaggle.com/c/google-football/submissions?dialog=episodes-submission-18348972

In [ ]:
import os
import json
import gc
import glob
import pickle
import sys
import time
import datetime

import numpy as np
import pandas as pd
import lightgbm as lgb
import math

from sklearn.model_selection import GroupKFold
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from tqdm import tqdm, notebook

pd.set_option('display.max_columns', 1000)

# Set Parameters
**for test**  
set `is_test_run`: True  
set `use_multi_data`: False 

**for main calculation**  
set `is_test_run`: False  
set `use_multi_data`: True (if necessary)  
set `skip_loss_game`: True (if necessary)  

In [ ]:
is_test_run = False

if is_test_run:
    op_learning_rate = 1
    op_early_stopping_rounds = 1
    op_lambda_l1 = 0
    op_lambda_l2 = 0
    max_num_leaves = 7
    op_feature_fraction = 0.1
    op_bagging_fraction = 0.1
    op_bagging_freq = 10
    op_min_child_samples = 100
    some_ramdom_state = 42
    skip_loss_game = False
else:
    op_learning_rate = 0.1
    op_early_stopping_rounds = 100
    op_lambda_l1 = 0.002639923022855581
    op_lambda_l2 = 4.299393968532123e-05
    max_num_leaves = 31
    op_feature_fraction = 0.98
    op_bagging_fraction = 0.98
    op_bagging_freq = 5
    op_min_child_samples = 10
    some_ramdom_state = 1638
    skip_loss_game = True

strong_agent = 18239675
data_source_dict = {18239675:'gf-get-episodes-18239675-nov28',
                    18127517:'gf-get-episodes-18127517-nov28'}
data_source = data_source_dict[strong_agent]
    
use_multi_data = False
if use_multi_data:
    if strong_agent == 18127517:
        strong_agent_2 = 18239675
    else:
        strong_agent_2 = 18127517
    data_source_2 = data_source_dict[strong_agent_2]
else:
    data_source_2 = ''    

# Functions for Feature Creation

In [ ]:
def convert_observation(observation):

    def do_flatten(obj):
        if type(obj) == list:
            return np.array(obj).flatten()
        return obj.flatten()
    
    def get_distance(pos1, pos2):
        return ((pos1[0]-pos2[0])**2 + (pos1[1]-pos2[1])**2)**0.5

    def get_distance_3D(pos1, pos2):
        return ((pos1[0]-pos2[0])**2 + (pos1[1]-pos2[1])**2 + (pos1[2]-pos2[2])**2)**0.5    
    
    def get_heading(pos1, pos2):
        return math.atan2(pos1[1]-pos2[1], pos1[0]-pos2[0])/math.pi*180 % 360

    def get_heading_3D(pos1, pos2):
        try:
            a = (pos1[2]-pos2[2])/get_distance_3D(pos1, pos2)
        except ZeroDivisionError:
            a = 0
        return math.acos(a)/math.pi*180   
    
    def get_cos_theta(vec1, vec2):
        inner_prod = (vec1[0]*vec2[0]) + (vec1[1]*vec2[1])
        norm = get_distance(vec1, (0, 0))*get_distance(vec2, (0, 0))
        return math.acos(inner_prod/norm)
    
    final_obs = []
    
    for obs in observation:
        #--------------------------#
        # observations on position #
        #--------------------------#
        o_pos = []
        for i, name in enumerate(['left_team', 'right_team']):
            o_pos.extend(do_flatten(obs[name]))
        # If there were less than 11vs11 players we backfill missing values
        # with -1.
            if len(o_pos) < (i + 1) * 22:
                o_pos.extend([-1] * ((i + 1) * 22 - len(o_pos)))

        o_dire = []
        for i, name in enumerate(['left_team_direction',
                                  'right_team_direction']):
            o_dire.extend(do_flatten(obs[name]))
        # If there were less than 11vs11 players we backfill missing values
        # with -1.
            if len(o_dire) < (i + 1) * 22:
                o_dire.extend([-1] * ((i + 1) * 22 - len(o_dire)))
                
        o_pos_center = []
        for i in range(len(o_pos)//2):
            o_pos_center.append(get_distance((o_pos[i*2],o_pos[i*2 + 1]), (0,0)))
            o_pos_center.append(get_heading((o_pos[i*2],o_pos[i*2 + 1]), (0,0)))
            
        o_dire_pt = []
        for i in range(len(o_dire)//2):
            o_dire_pt.append(get_distance((o_dire[i*2],o_dire[i*2 + 1]), (0,0)))
            o_dire_pt.append(get_heading((o_dire[i*2],o_dire[i*2 + 1]), (0,0)))
            
        o_pos_l_goal = []
        for i in range(len(o_pos)//2):
            o_pos_l_goal.append(get_distance((o_pos[i*2],o_pos[i*2 + 1]), (-1,0)))
            o_pos_l_goal.append(get_heading((o_pos[i*2],o_pos[i*2 + 1]), (-1,0)))

        o_pos_r_goal = []
        for i in range(len(o_pos)//2):
            o_pos_r_goal.append(get_distance((o_pos[i*2],o_pos[i*2 + 1]), (1,0)))
            o_pos_r_goal.append(get_heading((o_pos[i*2],o_pos[i*2 + 1]), (1,0)))            
            
        o_pos_activep = []
        activep = obs['active']
        for i in range(len(o_pos)//2):
            o_pos_activep.append(get_distance((o_pos[i*2],o_pos[i*2 + 1]),
                                             (o_pos[activep*2],o_pos[activep*2 + 1])))
            o_pos_activep.append(get_heading((o_pos[i*2],o_pos[i*2 + 1]),
                                            (o_pos[activep*2],o_pos[activep*2 + 1])))  

        o_pos_lp_near_activep = []
        lp_near_activep = int(np.concatenate([np.array(o_pos_activep).\
                                          flatten().reshape(-1, 2)[:activep-1, 0],\
                                          np.array(o_pos_activep).\
                                          flatten().reshape(-1, 2)[activep:11, 0]]).min())
        for i in range(len(o_pos)//2):
            o_pos_lp_near_activep.append(get_distance((o_pos[i*2],o_pos[i*2 + 1]),
                                             (o_pos[lp_near_activep*2],o_pos[lp_near_activep*2 + 1])))
            o_pos_lp_near_activep.append(get_heading((o_pos[i*2],o_pos[i*2 + 1]),
                                            (o_pos[lp_near_activep*2],o_pos[lp_near_activep*2 + 1])))

        o_pos_rp_near_activep = []
        rp_near_activep = int(np.array(o_pos_activep).flatten().reshape(-1, 2)[11:, 0].min())
        for i in range(len(o_pos)//2):
            o_pos_rp_near_activep.append(get_distance((o_pos[i*2],o_pos[i*2 + 1]),
                                             (o_pos[rp_near_activep*2],o_pos[rp_near_activep*2 + 1])))
            o_pos_rp_near_activep.append(get_heading((o_pos[i*2],o_pos[i*2 + 1]),
                                            (o_pos[rp_near_activep*2],o_pos[rp_near_activep*2 + 1])))
            
            
        o_pos_ball = []
        ball_xy = (obs['ball'][0], obs['ball'][1])
        for i in range(len(o_pos)//2):
            o_pos_ball.append(get_distance((o_pos[i*2],o_pos[i*2 + 1]), ball_xy))
            o_pos_ball.append(get_heading((o_pos[i*2],o_pos[i*2 + 1]), ball_xy))
        
        o_pos_rp_near_ball = []
        rp_near_ball = int(np.array(o_pos_ball).flatten().reshape(-1, 2)[11:, 0].min())
        for i in range(len(o_pos)//2):
            o_pos_rp_near_ball.append(get_distance((o_pos[i*2],o_pos[i*2 + 1]),
                                             (o_pos[rp_near_ball*2],o_pos[rp_near_ball*2 + 1])))
            o_pos_rp_near_ball.append(get_heading((o_pos[i*2],o_pos[i*2 + 1]),
                                            (o_pos[rp_near_ball*2],o_pos[rp_near_ball*2 + 1])))
            
        #--------------------------------------#
        # observations on state of the players #
        #--------------------------------------#

        o_state = []
        for i, name in enumerate(['left_team_tired_factor', 'left_team_yellow_card', 'left_team_roles',
                                'right_team_tired_factor', 'right_team_yellow_card', 'right_team_roles']):
            o_state.extend(do_flatten(obs[name]))
            if len(o_state) < (i + 1) * 11:
                o_state.extend([-1] * ((i + 1) * 11 - len(o_state)))

        #---------------------------#
        # observations on the ball  #
        #---------------------------#
        o_ball = []
        # ball position
        o_ball.extend(obs['ball'])
        
        o_ball_points = []
#         ball_xyz = (obs['ball'][0], obs['ball'][1], obs['ball'][2])
        ball_xyz = tuple(o_ball)
        for i, point in enumerate([(0, 0, 0), (-1, 0, 0), (1, 0, 0),
                                   (0, 0.42, 0), (0, -0.42, 0)]):
            o_ball_points.append(get_distance_3D(ball_xyz, point))
            o_ball_points.append(get_heading(ball_xyz, point))
            o_ball_points.append(get_heading_3D(ball_xyz, point))
            
        # ball direction
        o_ball_dire = []        
        o_ball_dire.extend(obs['ball_direction'])
        
        o_ball_dire_pt = []
        ball_dire_xyz = tuple(o_ball_dire)
        o_ball_dire_pt.append(get_distance_3D(ball_dire_xyz, (0, 0, 0)))        
        o_ball_dire_pt.append(get_heading(ball_dire_xyz, (0, 0, 0)))
        o_ball_dire_pt.append(get_heading_3D(ball_dire_xyz, (0, 0, 0)))

        # ball rotation -- It is not clear if the following calculation is physically valid.
        o_ball_rota = []           
        o_ball_rota.extend(obs['ball_rotation'])  
        
        o_ball_rota_pt = []
        ball_pota_xyz = tuple(o_ball_rota)
        o_ball_rota_pt.append(get_distance_3D(ball_pota_xyz, (0, 0, 0)))        
        o_ball_rota_pt.append(get_heading(ball_pota_xyz, (0, 0, 0)))
        o_ball_rota_pt.append(get_heading_3D(ball_pota_xyz, (0, 0, 0)))        
        
        o_ball_own = []
        o_ball_own.append(obs['ball_owned_team'])

        o_ball_angle_goal = []
        l_ball_goal = get_cos_theta((1-obs['ball'][0], 0.044-obs['ball'][1]), \
                                    (1-obs['ball'][0], -0.044-obs['ball'][1]))/math.pi
        l_ball_goal_var = l_ball_goal/get_distance((1,0), ball_xy)
        r_ball_goal = get_cos_theta((-1-obs['ball'][0], 0.044-obs['ball'][1]),\
                                    (-1-obs['ball'][0], -0.044-obs['ball'][1]))/math.pi
        r_ball_goal_var = r_ball_goal/get_distance((-1,0), ball_xy)

        o_ball_angle_goal.append(l_ball_goal)
        o_ball_angle_goal.append(l_ball_goal_var)
        o_ball_angle_goal.append(r_ball_goal)
        o_ball_angle_goal.append(r_ball_goal_var)
                               
        #-------------------------------#
        # observations on active player #
        #-------------------------------#          
        o_activep = []
        o_activep.append(obs['active'])      

        for o_list in [o_pos, o_dire, o_pos_center, o_dire_pt, o_pos_l_goal, o_pos_r_goal, o_pos_ball]:
            o_activep.append(o_list[activep*2])
            o_activep.append(o_list[activep*2 + 1])
            
        o_activep.append(o_state[activep])
        o_activep.append(o_state[activep + 11])
        o_activep.append(o_state[activep + 22])

        o_sticky = []
        o_sticky.extend(do_flatten(obs['sticky_actions']))
        
        #---------------------------#
        # observations on team      #
        #---------------------------#  
        o_team_grav = []
        for i, name in enumerate(['left_team', 'right_team']):
            temp_pos = do_flatten(obs[name])
            grav_x = 0
            grav_y = 0
            for i in range(len(temp_pos)//2):
                grav_x += temp_pos[i*2]
                grav_y += temp_pos[i*2 + 1]
            grav_x /= len(temp_pos)//2
            grav_y /= len(temp_pos)//2
        
            o_team_grav.append(grav_x)
            o_team_grav.append(grav_y)    

        o_team_grav.append((o_team_grav[0]+o_team_grav[2])/2)
        o_team_grav.append((o_team_grav[1]+o_team_grav[3])/2)
        
        o_team_grav_pt = []
        for i in range(len(o_team_grav)//2):
            o_team_grav_pt.append(get_distance((o_team_grav[i*2],o_team_grav[i*2 + 1]), (0,0)))
            o_team_grav_pt.append(get_heading((o_team_grav[i*2],o_team_grav[i*2 + 1]), (0,0)))
            
        o_pos_grav_left = []
        for i in range(len(o_pos)//2):
            o_pos_grav_left.append(get_distance((o_pos[i*2],o_pos[i*2 + 1]),
                                             (o_team_grav[0],o_team_grav[1])))
            o_pos_grav_left.append(get_heading((o_pos[i*2],o_pos[i*2 + 1]),
                                            (o_team_grav[0],o_team_grav[1])))  

        o_pos_grav_right = []
        for i in range(len(o_pos)//2):
            o_pos_grav_right.append(get_distance((o_pos[i*2],o_pos[i*2 + 1]),
                                             (o_team_grav[2],o_team_grav[3])))
            o_pos_grav_right.append(get_heading((o_pos[i*2],o_pos[i*2 + 1]),
                                            (o_team_grav[2],o_team_grav[3])))  
        o_pos_grav_both = []
        for i in range(len(o_pos)//2):
            o_pos_grav_both.append(get_distance((o_pos[i*2],o_pos[i*2 + 1]),
                                             (o_team_grav[4],o_team_grav[5])))
            o_pos_grav_both.append(get_heading((o_pos[i*2],o_pos[i*2 + 1]),
                                            (o_team_grav[4],o_team_grav[5])))  

        #---------------------------------#
        # observations on team without GK #
        #---------------------------------#  
        o_team_grav_woGK = []
        for i, name in enumerate(['left_team', 'right_team']):
            temp_pos = do_flatten(obs[name])
            grav_x = 0
            grav_y = 0
            for i in range(1, len(temp_pos)//2): # without GK
                grav_x += temp_pos[i*2]
                grav_y += temp_pos[i*2 + 1]
            grav_x /= len(temp_pos)//2
            grav_y /= len(temp_pos)//2
            o_team_grav_woGK.append(grav_x)
            o_team_grav_woGK.append(grav_y)    

        o_team_grav_woGK.append((o_team_grav_woGK[0]+o_team_grav_woGK[2])/2)
        o_team_grav_woGK.append((o_team_grav_woGK[1]+o_team_grav_woGK[3])/2)
        
        o_team_grav_woGK_pt = []
        for i in range(len(o_team_grav_woGK)//2):
            o_team_grav_woGK_pt.append(get_distance((o_team_grav_woGK[i*2],o_team_grav_woGK[i*2 + 1]), (0,0)))
            o_team_grav_woGK_pt.append(get_heading((o_team_grav_woGK[i*2],o_team_grav_woGK[i*2 + 1]), (0,0)))
            
        o_pos_grav_woGK_left = []
        for i in range(len(o_pos)//2):
            o_pos_grav_woGK_left.append(get_distance((o_pos[i*2],o_pos[i*2 + 1]),
                                             (o_team_grav_woGK[0],o_team_grav_woGK[1])))
            o_pos_grav_woGK_left.append(get_heading((o_pos[i*2],o_pos[i*2 + 1]),
                                            (o_team_grav_woGK[0],o_team_grav_woGK[1])))  

        o_pos_grav_woGK_right = []
        for i in range(len(o_pos)//2):
            o_pos_grav_woGK_right.append(get_distance((o_pos[i*2],o_pos[i*2 + 1]),
                                             (o_team_grav_woGK[2],o_team_grav_woGK[3])))
            o_pos_grav_woGK_right.append(get_heading((o_pos[i*2],o_pos[i*2 + 1]),
                                            (o_team_grav_woGK[2],o_team_grav_woGK[3])))  
        o_pos_grav_woGK_both = []
        for i in range(len(o_pos)//2):
            o_pos_grav_woGK_both.append(get_distance((o_pos[i*2],o_pos[i*2 + 1]),
                                             (o_team_grav_woGK[4],o_team_grav_woGK[5])))
            o_pos_grav_woGK_both.append(get_heading((o_pos[i*2],o_pos[i*2 + 1]),
                                            (o_team_grav_woGK[4],o_team_grav_woGK[5]))) 
        
        #-----------------------#
        # observations on line  #
        #-----------------------# 
        l_t_arr = np.array(obs['left_team']).flatten().reshape(-1, 2)
        
        o_pos_l_frontp = []
        l_frontp = l_t_arr[:,0].argmax()
        for i in range(len(o_pos)//2):
            o_pos_l_frontp.append(get_distance((o_pos[i*2],o_pos[i*2 + 1]),
                                             (o_pos[l_frontp*2],o_pos[l_frontp*2 + 1])))
            o_pos_l_frontp.append(get_heading((o_pos[i*2],o_pos[i*2 + 1]),
                                            (o_pos[l_frontp*2],o_pos[l_frontp*2 + 1])))          
        o_pos_l_backp = []
        l_backp = l_t_arr[1:,0].argmin() + 1
        for i in range(len(o_pos)//2):
            o_pos_l_backp.append(get_distance((o_pos[i*2],o_pos[i*2 + 1]),
                                             (o_pos[l_backp*2],o_pos[l_backp*2 + 1])))
            o_pos_l_backp.append(get_heading((o_pos[i*2],o_pos[i*2 + 1]),
                                            (o_pos[l_backp*2],o_pos[l_backp*2 + 1])))    

        r_t_arr = np.array(obs['right_team']).flatten().reshape(-1, 2)
        
        o_pos_r_frontp = []
        r_frontp = r_t_arr[:,0].argmin()
        for i in range(len(o_pos)//2):
            o_pos_r_frontp.append(get_distance((o_pos[i*2],o_pos[i*2 + 1]),
                                             (o_pos[22 + r_frontp*2],o_pos[22 + r_frontp*2 + 1])))
            o_pos_r_frontp.append(get_heading((o_pos[i*2],o_pos[i*2 + 1]),
                                            (o_pos[22 + r_frontp*2],o_pos[22 + r_frontp*2 + 1])))          
        o_pos_r_backp = []
        r_backp = r_t_arr[1:,0].argmax() + 1
        for i in range(len(o_pos)//2):
            o_pos_r_backp.append(get_distance((o_pos[i*2],o_pos[i*2 + 1]),
                                             (o_pos[22 + r_backp*2],o_pos[22 + r_backp*2 + 1])))
            o_pos_r_backp.append(get_heading((o_pos[i*2],o_pos[i*2 + 1]),
                                            (o_pos[22 + r_backp*2],o_pos[22 + r_backp*2 + 1])))  
  
        o_lines = []
        o_lines.append(o_pos[l_frontp*2])
        o_lines.append(o_pos[l_backp*2])
        o_lines.append(o_pos[22 + r_frontp*2])
        o_lines.append(o_pos[22 + r_backp*2])
        
        is_l_offside_pos = o_pos[l_frontp*2] > o_pos[22 + r_backp*2]
        o_lines.append(is_l_offside_pos)
        o_lines.append(is_l_offside_pos * o_pos[22 + r_backp*2])
        
        is_r_offside_pos = o_pos[l_backp*2] > o_pos[22 + r_frontp*2]
        o_lines.append(is_r_offside_pos)
        o_lines.append(is_r_offside_pos * o_pos[l_backp*2])

        o_width_x = []
        o_width_x.append(o_pos[l_frontp*2] - o_pos[l_backp*2])
        o_width_x.append(o_pos[l_frontp*2] - o_pos[22 + r_frontp*2])        
        o_width_x.append(o_pos[22 + r_backp*2] - o_pos[22 + r_frontp*2])
        o_width_x.append(o_pos[22 + r_backp*2] - o_pos[l_backp*2]) 
        
        o_width_y = []
        o_width_y.append(np.append(l_t_arr[l_t_arr[:,0] <= 0][:,1], -1).max())
        o_width_y.append(np.append(l_t_arr[l_t_arr[:,0] <= 0][:,1], 1).min())
        o_width_y.append(np.append(l_t_arr[l_t_arr[:,0] <= 0][:,1], -1).max()-\
                         np.append(l_t_arr[l_t_arr[:,0] <= 0][:,1], 1).min())
        o_width_y.append(np.append(l_t_arr[l_t_arr[:,0] > 0][:,1], -1).max())
        o_width_y.append(np.append(l_t_arr[l_t_arr[:,0] > 0][:,1], 1).min())
        o_width_y.append(np.append(l_t_arr[l_t_arr[:,0] > 0][:,1], -1).max()-\
                         np.append(l_t_arr[l_t_arr[:,0] > 0][:,1], 1).min())
        
        o_width_y.append(np.append(r_t_arr[r_t_arr[:,0] >= 0][:,1], -1).max())
        o_width_y.append(np.append(r_t_arr[r_t_arr[:,0] >= 0][:,1], 1).min())
        o_width_y.append(np.append(r_t_arr[r_t_arr[:,0] >= 0][:,1], -1).max()-\
                         np.append(r_t_arr[r_t_arr[:,0] >= 0][:,1], 1).min())
        o_width_y.append(np.append(r_t_arr[r_t_arr[:,0] < 0][:,1], -1).max())
        o_width_y.append(np.append(r_t_arr[r_t_arr[:,0] < 0][:,1], 1).min())
        o_width_y.append(np.append(r_t_arr[r_t_arr[:,0] < 0][:,1], -1).max()-\
                         np.append(r_t_arr[r_t_arr[:,0] < 0][:,1], 1).min()) 

        o_width_y.append(np.append(r_t_arr[:,1], -1).max())
        o_width_y.append(np.append(r_t_arr[:,1], 1).min())
        o_width_y.append(np.append(r_t_arr[:,1], -1).max()-\
                         np.append(r_t_arr[:,1], 1).min())
        o_width_y.append(np.append(r_t_arr[:,1], -1).max())
        o_width_y.append(np.append(r_t_arr[:,1], 1).min())
        o_width_y.append(np.append(r_t_arr[:,1], -1).max()-\
                         np.append(r_t_arr[:,1], 1).min()) 

        #-------------------------------------------#
        # observations on players reference of ball #
        #-------------------------------------------#
        o_ball_pnum = []
        for i, name in enumerate(['left_team', 'right_team']):
            temp_pos = do_flatten(obs[name])
            front_ball = 0
            back_ball = 0
            top_ball = 0
            bottom_ball = 0
            for i in range(len(temp_pos)//2):
                if temp_pos[i*2] > obs['ball'][0]:
                    front_ball += 1
                else:
                    back_ball += 1
                if temp_pos[i*2 + 1] > obs['ball'][1]:
                    top_ball += 1
                else:
                    bottom_ball += 1
            o_ball_pnum.append(front_ball)
            o_ball_pnum.append(back_ball)
            o_ball_pnum.append(top_ball)
            o_ball_pnum.append(bottom_ball)
                        
        o_ball_pnum.append(o_ball_pnum[0]+o_ball_pnum[4])
        o_ball_pnum.append(o_ball_pnum[1]+o_ball_pnum[5])
        o_ball_pnum.append(o_ball_pnum[2]+o_ball_pnum[6])
        o_ball_pnum.append(o_ball_pnum[3]+o_ball_pnum[7])
        
        #-------------------------------------------#
        # observations on around active_player/ball #
        #-------------------------------------------#
        
        o_pnum_around_activp = []
        for dist in [1, 3, 5, 10, 15]:
            pnum_dist_l = sum(np.array(o_pos_activep).flatten().reshape(-1, 2)[:11, 0] > 0.02 * dist)
            pnum_dist_r = sum(np.array(o_pos_activep).flatten().reshape(-1, 2)[11:, 0] > 0.02 * dist)
            pnum_dist_b = sum(np.array(o_pos_activep).flatten().reshape(-1, 2)[:, 0] > 0.02 * dist)        
            o_pnum_around_activp.append(pnum_dist_l)
            o_pnum_around_activp.append(pnum_dist_r)
            o_pnum_around_activp.append(pnum_dist_b)


        o_pnum_around_ball = []
        for dist in [1, 3, 5, 10, 15]:
            pnum_dist_l = sum(np.array(o_pos_ball).flatten().reshape(-1, 2)[:11, 0] > 0.02 * dist)
            pnum_dist_r = sum(np.array(o_pos_ball).flatten().reshape(-1, 2)[11:, 0] > 0.02 * dist)
            pnum_dist_b = sum(np.array(o_pos_ball).flatten().reshape(-1, 2)[:, 0] > 0.02 * dist)        
            o_pnum_around_ball.append(pnum_dist_l)
            o_pnum_around_ball.append(pnum_dist_r)
            o_pnum_around_ball.append(pnum_dist_b)
        
        #---------------------------#
        # observations on the other #
        #---------------------------#        
        o_other = []
    
        # o_other.extend(do_flatten(obs['score']))
        o_other.append(obs['score'][0])
        o_other.append(obs['score'][1])
        o_other.append(obs['score'][0] - obs['score'][1])
        
        o_other.append(obs['steps_left']) 
        
        o_other.append(obs['game_mode']) 

        #--------------------------#
        # contat observations      #
        #--------------------------#
        final_obs.extend(o_pos)
        final_obs.extend(o_dire)  
        final_obs.extend(o_pos_center)    
        final_obs.extend(o_dire_pt) 
        final_obs.extend(o_pos_l_goal) 
        final_obs.extend(o_pos_r_goal)         
        final_obs.extend(o_pos_activep)  
        final_obs.extend(o_pos_lp_near_activep) 
        final_obs.extend(o_pos_rp_near_activep) 
        final_obs.extend(o_pos_ball)
        final_obs.extend(o_pos_rp_near_ball)  
 
        final_obs.extend(o_state)   
    
        final_obs.extend(o_ball)     
        final_obs.extend(o_ball_points)           
        final_obs.extend(o_ball_dire)     
        final_obs.extend(o_ball_dire_pt)    
        final_obs.extend(o_ball_rota)     
        final_obs.extend(o_ball_rota_pt)    
        final_obs.extend(o_ball_own) 
        final_obs.extend(o_ball_angle_goal)                                        

        final_obs.extend(o_activep)        
        final_obs.extend(o_sticky) 

        final_obs.extend(o_team_grav)
        final_obs.extend(o_team_grav_pt)
        final_obs.extend(o_pos_grav_left)
        final_obs.extend(o_pos_grav_right)
        final_obs.extend(o_pos_grav_both)

        final_obs.extend(o_team_grav_woGK)
        final_obs.extend(o_team_grav_woGK_pt)
        final_obs.extend(o_pos_grav_woGK_left)
        final_obs.extend(o_pos_grav_woGK_right)
        final_obs.extend(o_pos_grav_woGK_both)
        
        final_obs.extend(o_pos_l_frontp)
        final_obs.extend(o_pos_l_backp)
        final_obs.extend(o_pos_r_frontp)
        final_obs.extend(o_pos_r_backp)
        final_obs.extend(o_lines)
        
        final_obs.extend(o_width_x)
        final_obs.extend(o_width_y)
        
        final_obs.extend(o_ball_pnum)
        
        final_obs.extend(o_pnum_around_activp)
        final_obs.extend(o_pnum_around_ball)
        
        final_obs.extend(o_other)
        
        return np.array(final_obs, dtype=np.float32).flatten()

In [ ]:
y=[]
x=[]
filenames = [p for p in os.listdir('/kaggle/input/'+data_source+'/episodes_log/') \
             if 'info' not in p and 'json' in p]

# for test
if is_test_run:
    filenames = filenames[:3]

for f in notebook.tqdm(filenames):
    filename = "/kaggle/input/"+data_source+"/episodes_log/" + f
    info_filename = "/kaggle/input/"+data_source+"/episodes_log/" + f.replace('.json', '_info.json')
    
    try:
        with open(filename) as json_file:
            data = json.load(json_file)
    except:
        continue
        
    try:
        with open(info_filename) as info_json_file:
            info = json.load(info_json_file)
    except:
        continue
        
        
    counter=0
    
    if info['agents'][0]['submissionId'] == strong_agent:
        team = 0
    else:
        team = 1
    
    for i in range(0,len(data['steps'])-2,):
        if skip_loss_game:
            reward_threshold = 0
        else:
            reward_threshold = -3000
            
        if data['steps'][-1][team]['reward'] >= reward_threshold:
            action=data['steps'][i+1][team]['action']
            y.append(action[0])

            obs=data['steps'][i][team]['observation']['players_raw'][0]
            x.append(convert_observation([obs]))

if use_multi_data:
    filenames = [p for p in os.listdir('/kaggle/input/'+data_source_2+'/episodes_log/') \
             if 'info' not in p and 'json' in p]

    # for test
    if is_test_run:
        filenames = filenames[:1]
    
    for f in notebook.tqdm(filenames):
        filename = "/kaggle/input/"+data_source_2+"/episodes_log/" + f
        info_filename = "/kaggle/input/"+data_source_2+"/episodes_log/" + f.replace('.json', '_info.json')

        try:
            with open(filename) as json_file:
                data = json.load(json_file)
        except:
            continue

        try:
            with open(info_filename) as info_json_file:
                info = json.load(info_json_file)
        except:
            continue


        counter=0

        if info['agents'][0]['submissionId'] == strong_agent_2:
            team = 0
        else:
            team = 1

        for i in range(0,len(data['steps'])-2,):
            if data['steps'][-1][team]['reward'] >= reward_threshold:
                action=data['steps'][i+1][team]['action']
                y.append(action[0])

                obs=data['steps'][i][team]['observation']['players_raw'][0]
                x.append(convert_observation([obs]))

In [ ]:
df_header = []

for name in ['left_team', 'left_team_direction', 'right_team', 'right_team_direction']:
    for i in list(range(11)):
        for xy in ['x', 'y']:
            df_header.append(name+'_'+xy+'_'+str(i))

for name in ['left_team', 'left_team_direction', 'right_team', 'right_team_direction']:
    for i in list(range(11)):
        for xy in ['r', 'theta']:
            df_header.append(name+'_'+xy+'_'+str(i))
            
for name in ['left_team_lg', 'right_team_lg', 'left_team_rg', 'right_team_rg',
             'left_team_ap', 'right_team_ap', \
             'left_team_l_near_ap', 'right_team_l_near_ap',\
             'left_team_r_near_ap', 'right_team_r_near_ap',\
             'left_team_ball', 'right_team_ball',\
             'left_team_r_near_ball', 'right_team_r_near_ball',]:
    for i in list(range(11)):
        for xy in ['r', 'theta']:
            df_header.append(name+'_'+xy+'_'+str(i))    
            
for name in ['left_team_tired_factor', 'left_team_yellow_card', 'left_team_roles',\
             'right_team_tired_factor', 'right_team_yellow_card', 'right_team_roles']:
    for i in list(range(11)):
        df_header.append(name+'_'+str(i))            

        
for name in ['ball']:
    for xyzrpt in ['x', 'y', 'z']:
         df_header.append(name+'_'+xyzrpt)
            
for name in ['ball_cp', 'ball_lg', 'ball_rg', 'ball_ct', 'ball_cb']:
    for rpt in ['r', 'phi', 'theta']:
         df_header.append(name+'_'+rpt)
            
for name in ['ball_direction', 'ball_rotation']:
    for xyzrpt in ['x', 'y', 'z', 'r', 'phi', 'theta']:
         df_header.append(name+'_'+xyzrpt)

df_header.append('ball_owned_team')

df_header.extend(['angle_l_ball_goal', 'var_l_ball_goal', \
                  'angle_r_ball_goal', 'var_r_ball_goal',]) \

df_header.append('active')

for name in ['x', 'y',  'direction_x', 'direction_y',
             'r', 'theta', 'direction_r', 'direction_theta',
             'lg_r', 'lg_theta', 'rg_r', 'rg_theta',
             'ball_r', 'ball_theta', 
             'tired_factor', 'yellow_card', 'roles']:
    df_header.append('left_team_activep_'+name)    

for name in ['left', 'top_left', 'top', 'top_right',
             'right', 'bottom_right', 'bottom', 'bottom_left',
             'sprint', 'dribble']:
    df_header.append('sticky_action_'+name) 

for name in ['left_team_grav', 'right_team_grav', 'both_team_grav']:
    for xy in ['x', 'y']:
        df_header.append(name+'_'+xy)

for name in ['left_team_grav', 'right_team_grav', 'both_team_grav']:
    for xy in ['r', 'theta']:
        df_header.append(name+'_'+xy)        
    
for name in ['left_team_l_grav', 'right_team_l_grav',
             'left_team_r_grav', 'right_team_r_grav',
             'left_team_b_grav', 'right_team_b_grav']:
    for i in list(range(11)):
        for xy in ['r', 'theta']:
            df_header.append(name+'_'+xy+'_'+str(i))
            
for name in ['left_team_grav_woGK', 'right_team_grav_woGK', 'both_team_grav_woGK']:
    for xy in ['x', 'y']:
        df_header.append(name+'_'+xy)

for name in ['left_team_grav_woGK', 'right_team_grav_woGK', 'both_team_grav_woGK']:
    for xy in ['r', 'theta']:
        df_header.append(name+'_'+xy)        
    
for name in ['left_team_l_grav_woGK', 'right_team_l_grav_woGK',
             'left_team_r_grav_woGK', 'right_team_r_grav_woGK',
             'left_team_b_grav_woGK', 'right_team_b_grav_woGK']:
    for i in list(range(11)):
        for xy in ['r', 'theta']:
            df_header.append(name+'_'+xy+'_'+str(i))
            
for name in ['left_team_l_fp', 'left_team_r_fp', 'left_team_l_bg', 'right_team_l_bg',
             'left_team_r_fp', 'right_team_r_fp','left_team_r_bg', 'right_team_r_bg']:
    for i in list(range(11)):
        for xy in ['r', 'theta']:
            df_header.append(name+'_'+xy+'_'+str(i))    

df_header.extend(['left_team_fp_x', 'left_team_bp_x', \
                  'right_team_fp_x', 'right_team_bp_x', \
                  'is_left_offside', 'left_offside_line', \
                  'is_right_offside', 'right_offside_line'])

df_header.extend(['width_x_l_fp_l_bp', 'width_x_l_fp_r_fp', \
                  'width_x_r_bp_r_fp', 'width_x_r_bp_l_bp'])

df_header.extend(['width_y_l_lc_max_y', 'width_y_l_lc_min_y', 'width_y_l_lc', \
                  'width_y_l_rc_max_y', 'width_y_l_rc_min_y', 'width_y_l_rc', \
                  'width_y_r_lc_max_y', 'width_y_r_lc_min_y', 'width_y_r_lc', \
                  'width_y_r_rc_max_y', 'width_y_r_rc_min_y', 'width_y_r_rc', \
                  'width_y_r_bc_max_y', 'width_y_b_lc_min_y', 'width_y_b_lc', \
                  'width_y_r_bc_max_y', 'width_y_b_rc_min_y', 'width_y_b_rc', \
                 ])

df_header.extend(['l_pnum_frb', 'l_pnum_bcb', 'l_pnum_tpb', 'l_pnum_btb',\
                  'r_pnum_frb', 'r_pnum_bcb', 'r_pnum_tpb', 'r_pnum_btb',\
                  'b_pnum_frb', 'b_pnum_bcb', 'b_pnum_tpb', 'b_pnum_btb',])

df_header.extend(['l_pnum_around_1_ap', 'r_pnum_around_1_ap', 'b_pnum_around_1_ap',\
                  'l_pnum_around_3_ap', 'r_pnum_around_3_ap', 'b_pnum_around_3_ap',\
                  'l_pnum_around_5_ap', 'r_pnum_around_5_ap', 'b_pnum_around_5_ap',\
                  'l_pnum_around_10_ap', 'r_pnum_around_10_ap', 'b_pnum_around_10_ap',\
                  'l_pnum_around_15_ap', 'r_pnum_around_15_ap', 'b_pnum_around_15_ap',])

df_header.extend(['l_pnum_around_1_ball', 'r_pnum_around_1_ball', 'b_pnum_around_1_ball',\
                  'l_pnum_around_3_ball', 'r_pnum_around_3_ball', 'b_pnum_around_3_ball',\
                  'l_pnum_around_5_ball', 'r_pnum_around_5_ball', 'b_pnum_around_5_ball',\
                  'l_pnum_around_10_ball', 'r_pnum_around_10_ball', 'b_pnum_around_10_ball',\
                  'l_pnum_around_15_ball', 'r_pnum_around_15_ball', 'b_pnum_around_15_ball',])

df_header.extend(['left_score', 'right_score', 'lead_score',\
                  'steps_left', 'game_mode'])

In [ ]:
len(df_header)

In [ ]:
df_x = pd.DataFrame(x, columns=df_header, dtype=np.float32)

In [ ]:
df_x.shape

In [ ]:
df_y = pd.Series(y, dtype=np.float32)

In [ ]:
cat_list = ['ball_owned_team', 'active', 'game_mode',
           'left_team_roles_0',
           'left_team_roles_1',
           'left_team_roles_2',
           'left_team_roles_3',
           'left_team_roles_4',
           'left_team_roles_5',
           'left_team_roles_6',
           'left_team_roles_7',
           'left_team_roles_8',
           'left_team_roles_9',
           'left_team_roles_10',
           'right_team_roles_0',
           'right_team_roles_1',
           'right_team_roles_2',
           'right_team_roles_3',
           'right_team_roles_4',
           'right_team_roles_5',
           'right_team_roles_6',
           'right_team_roles_7',
           'right_team_roles_8',
           'right_team_roles_9',
           'right_team_roles_10',
           'left_team_activep_roles'
           ]

In [ ]:
cat_list_num = []
for h in cat_list:
    cat_list_num.append(df_header.index(h))    

In [ ]:
gc.collect()

# Training LightGBM

In [ ]:
!mkdir -p /kaggle_simulations
!mkdir -p /kaggle_simulations/agent
!mkdir -p /kaggle_simulations/agent/saved_model

In [ ]:
df_xy = pd.concat([df_x, df_y], axis=1)
df_xy = df_xy.rename(columns={df_xy.columns[-1]: 'action'})

In [ ]:
for ball_own in [-1, 0, 1]:
    df_xy_bown = df_xy[df_xy['ball_owned_team'] == ball_own]
    gc.collect()
    
    df_weight = df_xy_bown['action'].map(np.log1p(1/df_xy_bown['action'].value_counts(normalize=True)/19))
    df_weight = df_weight * ((df_xy_bown['action'] != df_xy_bown['action'].shift(1)) * 0.5 + 1)
    
    
    X_train,X_test,y_train,y_test,w_train, w_test = \
      train_test_split(df_xy_bown.drop(['action'], axis=1),df_xy_bown['action'],df_weight,
                       test_size=0.2,
                       random_state=some_ramdom_state,
                       stratify=df_xy_bown['action'])
    gc.collect()
    lgb_train = lgb.Dataset(X_train,
                        label=y_train, 
                        categorical_feature=cat_list_num,
                        weight=w_train)
    gc.collect()
    lgb_eval = lgb.Dataset(X_test, 
                      label=y_test,
                      categorical_feature=cat_list_num,
#                       weight=X_test['weight'],
                      reference=lgb_train)
    gc.collect()
    params = {}
    params['objective'] = 'multiclass'
    params['num_classes'] = 19
    params['learning_rate'] = op_learning_rate
    params['ramdom_state'] = some_ramdom_state + int(ball_own)
    params['metric'] = 'multi_error'
    params['lambda_l1'] = op_lambda_l1
    params['lambda_l2'] = op_lambda_l2
    params['num_leaves'] = max_num_leaves
    params['feature_fraction'] = op_feature_fraction
    params['bagging_fraction'] = op_bagging_fraction
    params['bagging_freq'] = op_bagging_freq
    params['min_data_in_leaf'] = op_min_child_samples
    
    lgbmod = lgb.train(params, lgb_train, num_boost_round=10000,
               valid_sets=[lgb_eval], 
               early_stopping_rounds=op_early_stopping_rounds, 
               verbose_eval=100,)
    gc.collect()
    lgb.plot_importance(lgbmod, figsize=(12, 12), importance_type='gain', max_num_features=50)
    
    lgb.plot_importance(lgbmod, figsize=(12, 12), importance_type='split', max_num_features=50)
    
    if ball_own == -1:
        model_id = 'no'
    elif  ball_own == 0:
        model_id = 'lt'
    else:
        model_id = 'rt'
        
    
    filename = '/kaggle_simulations/agent/saved_model/lgbmod_'+model_id+'.pkl'
    pickle.dump(lgbmod, open(filename, 'wb'))
    
    preds = lgbmod.predict(X_test)
    best_preds = np.asarray([np.argmax(line) for line in preds])
    accuracy = accuracy_score(y_test, best_preds)
    gc.collect()     

# Make Agent

In [ ]:
%%writefile /kaggle_simulations/agent/main.py
from kaggle_environments.envs.football.helpers import *
import numpy as np
import pandas as pd
import lightgbm as lgb
import pickle
import math
from sklearn.metrics import accuracy_score
import glob

# load the model from disk
filename = '/kaggle_simulations/agent/saved_model/lgbmod_no.pkl'
loaded_model_no = pickle.load(open(filename, 'rb'))
filename = '/kaggle_simulations/agent/saved_model/lgbmod_lt.pkl'
loaded_model_lt = pickle.load(open(filename, 'rb'))
filename = '/kaggle_simulations/agent/saved_model/lgbmod_rt.pkl'
loaded_model_rt = pickle.load(open(filename, 'rb'))

def convert_observation(observation):

    def do_flatten(obj):
        if type(obj) == list:
            return np.array(obj).flatten()
        return obj.flatten()
    
    def get_distance(pos1, pos2):
        return ((pos1[0]-pos2[0])**2 + (pos1[1]-pos2[1])**2)**0.5

    def get_distance_3D(pos1, pos2):
        return ((pos1[0]-pos2[0])**2 + (pos1[1]-pos2[1])**2 + (pos1[2]-pos2[2])**2)**0.5    
    
    def get_heading(pos1, pos2):
        return math.atan2(pos1[1]-pos2[1], pos1[0]-pos2[0])/math.pi*180 % 360

    def get_heading_3D(pos1, pos2):
        try:
            a = (pos1[2]-pos2[2])/get_distance_3D(pos1, pos2)
        except ZeroDivisionError:
            a = 0
        return math.acos(a)/math.pi*180   
    
    def get_cos_theta(vec1, vec2):
        inner_prod = (vec1[0]*vec2[0]) + (vec1[1]*vec2[1])
        norm = get_distance(vec1, (0, 0))*get_distance(vec2, (0, 0))
        return math.acos(inner_prod/norm)
    
    final_obs = []
    
    for obs in observation:
        #--------------------------#
        # observations on position #
        #--------------------------#
        o_pos = []
        for i, name in enumerate(['left_team', 'right_team']):
            o_pos.extend(do_flatten(obs[name]))
        # If there were less than 11vs11 players we backfill missing values
        # with -1.
            if len(o_pos) < (i + 1) * 22:
                o_pos.extend([-1] * ((i + 1) * 22 - len(o_pos)))

        o_dire = []
        for i, name in enumerate(['left_team_direction',
                                  'right_team_direction']):
            o_dire.extend(do_flatten(obs[name]))
        # If there were less than 11vs11 players we backfill missing values
        # with -1.
            if len(o_dire) < (i + 1) * 22:
                o_dire.extend([-1] * ((i + 1) * 22 - len(o_dire)))
                
        o_pos_center = []
        for i in range(len(o_pos)//2):
            o_pos_center.append(get_distance((o_pos[i*2],o_pos[i*2 + 1]), (0,0)))
            o_pos_center.append(get_heading((o_pos[i*2],o_pos[i*2 + 1]), (0,0)))
            
        o_dire_pt = []
        for i in range(len(o_dire)//2):
            o_dire_pt.append(get_distance((o_dire[i*2],o_dire[i*2 + 1]), (0,0)))
            o_dire_pt.append(get_heading((o_dire[i*2],o_dire[i*2 + 1]), (0,0)))
            
        o_pos_l_goal = []
        for i in range(len(o_pos)//2):
            o_pos_l_goal.append(get_distance((o_pos[i*2],o_pos[i*2 + 1]), (-1,0)))
            o_pos_l_goal.append(get_heading((o_pos[i*2],o_pos[i*2 + 1]), (-1,0)))

        o_pos_r_goal = []
        for i in range(len(o_pos)//2):
            o_pos_r_goal.append(get_distance((o_pos[i*2],o_pos[i*2 + 1]), (1,0)))
            o_pos_r_goal.append(get_heading((o_pos[i*2],o_pos[i*2 + 1]), (1,0)))            
            
        o_pos_activep = []
        activep = obs['active']
        for i in range(len(o_pos)//2):
            o_pos_activep.append(get_distance((o_pos[i*2],o_pos[i*2 + 1]),
                                             (o_pos[activep*2],o_pos[activep*2 + 1])))
            o_pos_activep.append(get_heading((o_pos[i*2],o_pos[i*2 + 1]),
                                            (o_pos[activep*2],o_pos[activep*2 + 1])))  

        o_pos_lp_near_activep = []
        lp_near_activep = int(np.concatenate([np.array(o_pos_activep).\
                                          flatten().reshape(-1, 2)[:activep-1, 0],\
                                          np.array(o_pos_activep).\
                                          flatten().reshape(-1, 2)[activep:11, 0]]).min())
        for i in range(len(o_pos)//2):
            o_pos_lp_near_activep.append(get_distance((o_pos[i*2],o_pos[i*2 + 1]),
                                             (o_pos[lp_near_activep*2],o_pos[lp_near_activep*2 + 1])))
            o_pos_lp_near_activep.append(get_heading((o_pos[i*2],o_pos[i*2 + 1]),
                                            (o_pos[lp_near_activep*2],o_pos[lp_near_activep*2 + 1])))

        o_pos_rp_near_activep = []
        rp_near_activep = int(np.array(o_pos_activep).flatten().reshape(-1, 2)[11:, 0].min())
        for i in range(len(o_pos)//2):
            o_pos_rp_near_activep.append(get_distance((o_pos[i*2],o_pos[i*2 + 1]),
                                             (o_pos[rp_near_activep*2],o_pos[rp_near_activep*2 + 1])))
            o_pos_rp_near_activep.append(get_heading((o_pos[i*2],o_pos[i*2 + 1]),
                                            (o_pos[rp_near_activep*2],o_pos[rp_near_activep*2 + 1])))
            
            
        o_pos_ball = []
        ball_xy = (obs['ball'][0], obs['ball'][1])
        for i in range(len(o_pos)//2):
            o_pos_ball.append(get_distance((o_pos[i*2],o_pos[i*2 + 1]), ball_xy))
            o_pos_ball.append(get_heading((o_pos[i*2],o_pos[i*2 + 1]), ball_xy))
        
        o_pos_rp_near_ball = []
        rp_near_ball = int(np.array(o_pos_ball).flatten().reshape(-1, 2)[11:, 0].min())
        for i in range(len(o_pos)//2):
            o_pos_rp_near_ball.append(get_distance((o_pos[i*2],o_pos[i*2 + 1]),
                                             (o_pos[rp_near_ball*2],o_pos[rp_near_ball*2 + 1])))
            o_pos_rp_near_ball.append(get_heading((o_pos[i*2],o_pos[i*2 + 1]),
                                            (o_pos[rp_near_ball*2],o_pos[rp_near_ball*2 + 1])))
            
        #--------------------------------------#
        # observations on state of the players #
        #--------------------------------------#

        o_state = []
        for i, name in enumerate(['left_team_tired_factor', 'left_team_yellow_card', 'left_team_roles',
                                'right_team_tired_factor', 'right_team_yellow_card', 'right_team_roles']):
            o_state.extend(do_flatten(obs[name]))
            if len(o_state) < (i + 1) * 11:
                o_state.extend([-1] * ((i + 1) * 11 - len(o_state)))

        #---------------------------#
        # observations on the ball  #
        #---------------------------#
        o_ball = []
        # ball position
        o_ball.extend(obs['ball'])
        
        o_ball_points = []
#         ball_xyz = (obs['ball'][0], obs['ball'][1], obs['ball'][2])
        ball_xyz = tuple(o_ball)
        for i, point in enumerate([(0, 0, 0), (-1, 0, 0), (1, 0, 0),
                                   (0, 0.42, 0), (0, -0.42, 0)]):
            o_ball_points.append(get_distance_3D(ball_xyz, point))
            o_ball_points.append(get_heading(ball_xyz, point))
            o_ball_points.append(get_heading_3D(ball_xyz, point))
            
        # ball direction
        o_ball_dire = []        
        o_ball_dire.extend(obs['ball_direction'])
        
        o_ball_dire_pt = []
        ball_dire_xyz = tuple(o_ball_dire)
        o_ball_dire_pt.append(get_distance_3D(ball_dire_xyz, (0, 0, 0)))        
        o_ball_dire_pt.append(get_heading(ball_dire_xyz, (0, 0, 0)))
        o_ball_dire_pt.append(get_heading_3D(ball_dire_xyz, (0, 0, 0)))

        # ball rotation -- It is not clear if the following calculation is physically valid.
        o_ball_rota = []           
        o_ball_rota.extend(obs['ball_rotation'])  
        
        o_ball_rota_pt = []
        ball_pota_xyz = tuple(o_ball_rota)
        o_ball_rota_pt.append(get_distance_3D(ball_pota_xyz, (0, 0, 0)))        
        o_ball_rota_pt.append(get_heading(ball_pota_xyz, (0, 0, 0)))
        o_ball_rota_pt.append(get_heading_3D(ball_pota_xyz, (0, 0, 0)))        
        
        o_ball_own = []
        o_ball_own.append(obs['ball_owned_team'])

        o_ball_angle_goal = []
        l_ball_goal = get_cos_theta((1-obs['ball'][0], 0.044-obs['ball'][1]), \
                                    (1-obs['ball'][0], -0.044-obs['ball'][1]))/math.pi
        l_ball_goal_var = l_ball_goal/get_distance((1,0), ball_xy)
        r_ball_goal = get_cos_theta((-1-obs['ball'][0], 0.044-obs['ball'][1]),\
                                    (-1-obs['ball'][0], -0.044-obs['ball'][1]))/math.pi
        r_ball_goal_var = r_ball_goal/get_distance((-1,0), ball_xy)

        o_ball_angle_goal.append(l_ball_goal)
        o_ball_angle_goal.append(l_ball_goal_var)
        o_ball_angle_goal.append(r_ball_goal)
        o_ball_angle_goal.append(r_ball_goal_var)
                               
        #-------------------------------#
        # observations on active player #
        #-------------------------------#          
        o_activep = []
        o_activep.append(obs['active'])      

        for o_list in [o_pos, o_dire, o_pos_center, o_dire_pt, o_pos_l_goal, o_pos_r_goal, o_pos_ball]:
            o_activep.append(o_list[activep*2])
            o_activep.append(o_list[activep*2 + 1])
            
        o_activep.append(o_state[activep])
        o_activep.append(o_state[activep + 11])
        o_activep.append(o_state[activep + 22])

        o_sticky = []
        o_sticky.extend(do_flatten(obs['sticky_actions']))
        
        #---------------------------#
        # observations on team      #
        #---------------------------#  
        o_team_grav = []
        for i, name in enumerate(['left_team', 'right_team']):
            temp_pos = do_flatten(obs[name])
            grav_x = 0
            grav_y = 0
            for i in range(len(temp_pos)//2):
                grav_x += temp_pos[i*2]
                grav_y += temp_pos[i*2 + 1]
            grav_x /= len(temp_pos)//2
            grav_y /= len(temp_pos)//2
        
            o_team_grav.append(grav_x)
            o_team_grav.append(grav_y)    

        o_team_grav.append((o_team_grav[0]+o_team_grav[2])/2)
        o_team_grav.append((o_team_grav[1]+o_team_grav[3])/2)
        
        o_team_grav_pt = []
        for i in range(len(o_team_grav)//2):
            o_team_grav_pt.append(get_distance((o_team_grav[i*2],o_team_grav[i*2 + 1]), (0,0)))
            o_team_grav_pt.append(get_heading((o_team_grav[i*2],o_team_grav[i*2 + 1]), (0,0)))
            
        o_pos_grav_left = []
        for i in range(len(o_pos)//2):
            o_pos_grav_left.append(get_distance((o_pos[i*2],o_pos[i*2 + 1]),
                                             (o_team_grav[0],o_team_grav[1])))
            o_pos_grav_left.append(get_heading((o_pos[i*2],o_pos[i*2 + 1]),
                                            (o_team_grav[0],o_team_grav[1])))  

        o_pos_grav_right = []
        for i in range(len(o_pos)//2):
            o_pos_grav_right.append(get_distance((o_pos[i*2],o_pos[i*2 + 1]),
                                             (o_team_grav[2],o_team_grav[3])))
            o_pos_grav_right.append(get_heading((o_pos[i*2],o_pos[i*2 + 1]),
                                            (o_team_grav[2],o_team_grav[3])))  
        o_pos_grav_both = []
        for i in range(len(o_pos)//2):
            o_pos_grav_both.append(get_distance((o_pos[i*2],o_pos[i*2 + 1]),
                                             (o_team_grav[4],o_team_grav[5])))
            o_pos_grav_both.append(get_heading((o_pos[i*2],o_pos[i*2 + 1]),
                                            (o_team_grav[4],o_team_grav[5])))  

        #---------------------------------#
        # observations on team without GK #
        #---------------------------------#  
        o_team_grav_woGK = []
        for i, name in enumerate(['left_team', 'right_team']):
            temp_pos = do_flatten(obs[name])
            grav_x = 0
            grav_y = 0
            for i in range(1, len(temp_pos)//2): # without GK
                grav_x += temp_pos[i*2]
                grav_y += temp_pos[i*2 + 1]
            grav_x /= len(temp_pos)//2
            grav_y /= len(temp_pos)//2
            o_team_grav_woGK.append(grav_x)
            o_team_grav_woGK.append(grav_y)    

        o_team_grav_woGK.append((o_team_grav_woGK[0]+o_team_grav_woGK[2])/2)
        o_team_grav_woGK.append((o_team_grav_woGK[1]+o_team_grav_woGK[3])/2)
        
        o_team_grav_woGK_pt = []
        for i in range(len(o_team_grav_woGK)//2):
            o_team_grav_woGK_pt.append(get_distance((o_team_grav_woGK[i*2],o_team_grav_woGK[i*2 + 1]), (0,0)))
            o_team_grav_woGK_pt.append(get_heading((o_team_grav_woGK[i*2],o_team_grav_woGK[i*2 + 1]), (0,0)))
            
        o_pos_grav_woGK_left = []
        for i in range(len(o_pos)//2):
            o_pos_grav_woGK_left.append(get_distance((o_pos[i*2],o_pos[i*2 + 1]),
                                             (o_team_grav_woGK[0],o_team_grav_woGK[1])))
            o_pos_grav_woGK_left.append(get_heading((o_pos[i*2],o_pos[i*2 + 1]),
                                            (o_team_grav_woGK[0],o_team_grav_woGK[1])))  

        o_pos_grav_woGK_right = []
        for i in range(len(o_pos)//2):
            o_pos_grav_woGK_right.append(get_distance((o_pos[i*2],o_pos[i*2 + 1]),
                                             (o_team_grav_woGK[2],o_team_grav_woGK[3])))
            o_pos_grav_woGK_right.append(get_heading((o_pos[i*2],o_pos[i*2 + 1]),
                                            (o_team_grav_woGK[2],o_team_grav_woGK[3])))  
        o_pos_grav_woGK_both = []
        for i in range(len(o_pos)//2):
            o_pos_grav_woGK_both.append(get_distance((o_pos[i*2],o_pos[i*2 + 1]),
                                             (o_team_grav_woGK[4],o_team_grav_woGK[5])))
            o_pos_grav_woGK_both.append(get_heading((o_pos[i*2],o_pos[i*2 + 1]),
                                            (o_team_grav_woGK[4],o_team_grav_woGK[5]))) 
        
        #-----------------------#
        # observations on line  #
        #-----------------------# 
        l_t_arr = np.array(obs['left_team']).flatten().reshape(-1, 2)
        
        o_pos_l_frontp = []
        l_frontp = l_t_arr[:,0].argmax()
        for i in range(len(o_pos)//2):
            o_pos_l_frontp.append(get_distance((o_pos[i*2],o_pos[i*2 + 1]),
                                             (o_pos[l_frontp*2],o_pos[l_frontp*2 + 1])))
            o_pos_l_frontp.append(get_heading((o_pos[i*2],o_pos[i*2 + 1]),
                                            (o_pos[l_frontp*2],o_pos[l_frontp*2 + 1])))          
        o_pos_l_backp = []
        l_backp = l_t_arr[1:,0].argmin() + 1
        for i in range(len(o_pos)//2):
            o_pos_l_backp.append(get_distance((o_pos[i*2],o_pos[i*2 + 1]),
                                             (o_pos[l_backp*2],o_pos[l_backp*2 + 1])))
            o_pos_l_backp.append(get_heading((o_pos[i*2],o_pos[i*2 + 1]),
                                            (o_pos[l_backp*2],o_pos[l_backp*2 + 1])))    

        r_t_arr = np.array(obs['right_team']).flatten().reshape(-1, 2)
        
        o_pos_r_frontp = []
        r_frontp = r_t_arr[:,0].argmin()
        for i in range(len(o_pos)//2):
            o_pos_r_frontp.append(get_distance((o_pos[i*2],o_pos[i*2 + 1]),
                                             (o_pos[22 + r_frontp*2],o_pos[22 + r_frontp*2 + 1])))
            o_pos_r_frontp.append(get_heading((o_pos[i*2],o_pos[i*2 + 1]),
                                            (o_pos[22 + r_frontp*2],o_pos[22 + r_frontp*2 + 1])))          
        o_pos_r_backp = []
        r_backp = r_t_arr[1:,0].argmax() + 1
        for i in range(len(o_pos)//2):
            o_pos_r_backp.append(get_distance((o_pos[i*2],o_pos[i*2 + 1]),
                                             (o_pos[22 + r_backp*2],o_pos[22 + r_backp*2 + 1])))
            o_pos_r_backp.append(get_heading((o_pos[i*2],o_pos[i*2 + 1]),
                                            (o_pos[22 + r_backp*2],o_pos[22 + r_backp*2 + 1])))  
  
        o_lines = []
        o_lines.append(o_pos[l_frontp*2])
        o_lines.append(o_pos[l_backp*2])
        o_lines.append(o_pos[22 + r_frontp*2])
        o_lines.append(o_pos[22 + r_backp*2])
        
        is_l_offside_pos = o_pos[l_frontp*2] > o_pos[22 + r_backp*2]
        o_lines.append(is_l_offside_pos)
        o_lines.append(is_l_offside_pos * o_pos[22 + r_backp*2])
        
        is_r_offside_pos = o_pos[l_backp*2] > o_pos[22 + r_frontp*2]
        o_lines.append(is_r_offside_pos)
        o_lines.append(is_r_offside_pos * o_pos[l_backp*2])

        o_width_x = []
        o_width_x.append(o_pos[l_frontp*2] - o_pos[l_backp*2])
        o_width_x.append(o_pos[l_frontp*2] - o_pos[22 + r_frontp*2])        
        o_width_x.append(o_pos[22 + r_backp*2] - o_pos[22 + r_frontp*2])
        o_width_x.append(o_pos[22 + r_backp*2] - o_pos[l_backp*2]) 
        
        o_width_y = []
        o_width_y.append(np.append(l_t_arr[l_t_arr[:,0] <= 0][:,1], -1).max())
        o_width_y.append(np.append(l_t_arr[l_t_arr[:,0] <= 0][:,1], 1).min())
        o_width_y.append(np.append(l_t_arr[l_t_arr[:,0] <= 0][:,1], -1).max()-\
                         np.append(l_t_arr[l_t_arr[:,0] <= 0][:,1], 1).min())
        o_width_y.append(np.append(l_t_arr[l_t_arr[:,0] > 0][:,1], -1).max())
        o_width_y.append(np.append(l_t_arr[l_t_arr[:,0] > 0][:,1], 1).min())
        o_width_y.append(np.append(l_t_arr[l_t_arr[:,0] > 0][:,1], -1).max()-\
                         np.append(l_t_arr[l_t_arr[:,0] > 0][:,1], 1).min())
        
        o_width_y.append(np.append(r_t_arr[r_t_arr[:,0] >= 0][:,1], -1).max())
        o_width_y.append(np.append(r_t_arr[r_t_arr[:,0] >= 0][:,1], 1).min())
        o_width_y.append(np.append(r_t_arr[r_t_arr[:,0] >= 0][:,1], -1).max()-\
                         np.append(r_t_arr[r_t_arr[:,0] >= 0][:,1], 1).min())
        o_width_y.append(np.append(r_t_arr[r_t_arr[:,0] < 0][:,1], -1).max())
        o_width_y.append(np.append(r_t_arr[r_t_arr[:,0] < 0][:,1], 1).min())
        o_width_y.append(np.append(r_t_arr[r_t_arr[:,0] < 0][:,1], -1).max()-\
                         np.append(r_t_arr[r_t_arr[:,0] < 0][:,1], 1).min()) 

        o_width_y.append(np.append(r_t_arr[:,1], -1).max())
        o_width_y.append(np.append(r_t_arr[:,1], 1).min())
        o_width_y.append(np.append(r_t_arr[:,1], -1).max()-\
                         np.append(r_t_arr[:,1], 1).min())
        o_width_y.append(np.append(r_t_arr[:,1], -1).max())
        o_width_y.append(np.append(r_t_arr[:,1], 1).min())
        o_width_y.append(np.append(r_t_arr[:,1], -1).max()-\
                         np.append(r_t_arr[:,1], 1).min()) 

        #-------------------------------------------#
        # observations on players reference of ball #
        #-------------------------------------------#
        o_ball_pnum = []
        for i, name in enumerate(['left_team', 'right_team']):
            temp_pos = do_flatten(obs[name])
            front_ball = 0
            back_ball = 0
            top_ball = 0
            bottom_ball = 0
            for i in range(len(temp_pos)//2):
                if temp_pos[i*2] > obs['ball'][0]:
                    front_ball += 1
                else:
                    back_ball += 1
                if temp_pos[i*2 + 1] > obs['ball'][1]:
                    top_ball += 1
                else:
                    bottom_ball += 1
            o_ball_pnum.append(front_ball)
            o_ball_pnum.append(back_ball)
            o_ball_pnum.append(top_ball)
            o_ball_pnum.append(bottom_ball)
                        
        o_ball_pnum.append(o_ball_pnum[0]+o_ball_pnum[4])
        o_ball_pnum.append(o_ball_pnum[1]+o_ball_pnum[5])
        o_ball_pnum.append(o_ball_pnum[2]+o_ball_pnum[6])
        o_ball_pnum.append(o_ball_pnum[3]+o_ball_pnum[7])
        
        #-------------------------------------------#
        # observations on around active_player/ball #
        #-------------------------------------------#
        
        o_pnum_around_activp = []
        for dist in [1, 3, 5, 10, 15]:
            pnum_dist_l = sum(np.array(o_pos_activep).flatten().reshape(-1, 2)[:11, 0] > 0.02 * dist)
            pnum_dist_r = sum(np.array(o_pos_activep).flatten().reshape(-1, 2)[11:, 0] > 0.02 * dist)
            pnum_dist_b = sum(np.array(o_pos_activep).flatten().reshape(-1, 2)[:, 0] > 0.02 * dist)        
            o_pnum_around_activp.append(pnum_dist_l)
            o_pnum_around_activp.append(pnum_dist_r)
            o_pnum_around_activp.append(pnum_dist_b)


        o_pnum_around_ball = []
        for dist in [1, 3, 5, 10, 15]:
            pnum_dist_l = sum(np.array(o_pos_ball).flatten().reshape(-1, 2)[:11, 0] > 0.02 * dist)
            pnum_dist_r = sum(np.array(o_pos_ball).flatten().reshape(-1, 2)[11:, 0] > 0.02 * dist)
            pnum_dist_b = sum(np.array(o_pos_ball).flatten().reshape(-1, 2)[:, 0] > 0.02 * dist)        
            o_pnum_around_ball.append(pnum_dist_l)
            o_pnum_around_ball.append(pnum_dist_r)
            o_pnum_around_ball.append(pnum_dist_b)
        
        
        
        #---------------------------#
        # observations on the other #
        #---------------------------#        
        o_other = []
    
        # o_other.extend(do_flatten(obs['score']))
        o_other.append(obs['score'][0])
        o_other.append(obs['score'][1])
        o_other.append(obs['score'][0] - obs['score'][1])
        
        o_other.append(obs['steps_left']) 
        
        o_other.append(obs['game_mode']) 


        
        #--------------------------#
        # contat observations      #
        #--------------------------#
        final_obs.extend(o_pos)
        final_obs.extend(o_dire)  
        final_obs.extend(o_pos_center)    
        final_obs.extend(o_dire_pt) 
        final_obs.extend(o_pos_l_goal) 
        final_obs.extend(o_pos_r_goal)         
        final_obs.extend(o_pos_activep)  
        final_obs.extend(o_pos_lp_near_activep) 
        final_obs.extend(o_pos_rp_near_activep) 
        final_obs.extend(o_pos_ball)
        final_obs.extend(o_pos_rp_near_ball)  
 
        final_obs.extend(o_state)   
    
        final_obs.extend(o_ball)     
        final_obs.extend(o_ball_points)           
        final_obs.extend(o_ball_dire)     
        final_obs.extend(o_ball_dire_pt)    
        final_obs.extend(o_ball_rota)     
        final_obs.extend(o_ball_rota_pt)    
        final_obs.extend(o_ball_own) 
        final_obs.extend(o_ball_angle_goal)                                        

        final_obs.extend(o_activep)        
        final_obs.extend(o_sticky) 

        final_obs.extend(o_team_grav)
        final_obs.extend(o_team_grav_pt)
        final_obs.extend(o_pos_grav_left)
        final_obs.extend(o_pos_grav_right)
        final_obs.extend(o_pos_grav_both)

        final_obs.extend(o_team_grav_woGK)
        final_obs.extend(o_team_grav_woGK_pt)
        final_obs.extend(o_pos_grav_woGK_left)
        final_obs.extend(o_pos_grav_woGK_right)
        final_obs.extend(o_pos_grav_woGK_both)
        
        final_obs.extend(o_pos_l_frontp)
        final_obs.extend(o_pos_l_backp)
        final_obs.extend(o_pos_r_frontp)
        final_obs.extend(o_pos_r_backp)
        final_obs.extend(o_lines)
        
        final_obs.extend(o_width_x)
        final_obs.extend(o_width_y)
        
        final_obs.extend(o_ball_pnum)
        
        final_obs.extend(o_pnum_around_activp)
        final_obs.extend(o_pnum_around_ball)
        
        final_obs.extend(o_other)
            
        return np.array(final_obs, dtype=np.float32).flatten()

# @human_readable_agent
def agent(obs):
    try:
        obs1=obs['observation']['players_raw'][0]
    except:
        obs1=obs['players_raw'][0]
        
    conv_obs1=convert_observation([obs1])
    
    # predict
    if obs1['ball_owned_team'] == 0:
        action=np.argmax(loaded_model_lt.predict([conv_obs1]).flatten())
    elif obs1['ball_owned_team'] == 1:
        action=np.argmax(loaded_model_rt.predict([conv_obs1]).flatten())
    else:
        action=np.argmax(loaded_model_no.predict([conv_obs1]).flatten())
    
    return [int(action)]

In [ ]:
!cd /kaggle_simulations/agent && tar -czvf /kaggle/working/submit.tar.gz main.py saved_model

In [ ]:
!tar -zxvf ./submit.tar.gz

# Check Agent Operation

In [ ]:
# Install:
# Kaggle environments.
!git clone https://github.com/Kaggle/kaggle-environments.git
!cd kaggle-environments && pip install .

# GFootball environment.
!apt-get update -y
!apt-get install -y libsdl2-gfx-dev libsdl2-ttf-dev

# Make sure that the Branch in git clone and in wget call matches !!
!git clone -b v2.7 https://github.com/google-research/football.git
!mkdir -p football/third_party/gfootball_engine/lib

!wget https://storage.googleapis.com/gfootball/prebuilt_gameplayfootball_v2.7.so -O football/third_party/gfootball_engine/lib/prebuilt_gameplayfootball.so
!cd football && GFOOTBALL_USE_PREBUILT_SO=1 pip3 install .

In [ ]:
from kaggle_environments import make

env = make("football", configuration={"save_video": True, "scenario_name": "11_vs_11_kaggle", "running_in_notebook": True})
output = env.run(["/kaggle/working/main.py",
                  "/kaggle/working/main.py"])
print('Left player: reward = %s, status = %s, info = %s' % (output[-1][0]['reward'],
                                                            output[-1][0]['status'], output[-1][0]['info']))
env.render(mode='human',width=800, height=600)

In [ ]:
!rm -rf ./football
!rm -rf ./kaggle-environments